# Compare output to particles

Compare the stats outputs to those calculated from ParticleGroup

In [ ]:
import numpy as np
import impact.z as IZ
from impact.z import WriteFull, Drift, ImpactZOutput

from pmd_beamphysics import ParticleGroup
from pmd_beamphysics.units import mec2

from scipy.constants import c

In [ ]:
energy0 = 10e6
mc2 = mec2
n_particle = 100_000

input = IZ.ImpactZInput(
    initial_particles=None,
    ncpu_y=1,
    ncpu_z=1,
    # gpu=<GPUFlag.disabled: 0>,
    seed=-1,
    n_particle=n_particle,
    integrator_type=1,
    err=1,
    diagnostic_type=1,
    # output_z=2,
    nx=64,
    ny=64,
    nz=64,
    boundary_type=1,
    radius_x=0.0,
    radius_y=0.0,
    z_period_size=0.0,
    distribution="gauss",
    restart=0,
    subcycle=0,
    nbunch=0,
    particle_list=[0],
    current_list=[0.0],
    charge_over_mass_list=[0.0],
    twiss_alpha_x=1,
    twiss_beta_x=10.0,
    twiss_norm_emit_x=1e-06,
    twiss_mismatch_x=1.0,
    twiss_mismatch_px=1.0,
    twiss_offset_x=0,
    twiss_offset_px=0.0,
    twiss_alpha_y=0.0,
    twiss_beta_y=10.0,
    twiss_norm_emit_y=1e-06,
    twiss_mismatch_y=1.0,
    twiss_mismatch_py=1.0,
    twiss_offset_y=0.0,
    twiss_offset_py=0.0,
    twiss_alpha_z=1,
    twiss_beta_z=3.5999999999999996,
    twiss_norm_emit_z=0.036,
    twiss_mismatch_z=1.0,
    twiss_mismatch_e_z=1.0,
    twiss_offset_phase_z=0.0,
    twiss_offset_energy_z=-2,
    average_current=1,
    reference_kinetic_energy=energy0 - mc2,
    reference_particle_mass=mc2,
    reference_particle_charge=-1.0,
    reference_frequency=1e9,
    initial_phase_ref=0.0,
    lattice=[
        WriteFull(
            name="initial_particles",
            length=0.0,
            steps=0,
            file_id=100,
            type_id=-2,
            unused_2=0.0,
            sample_frequency=0,
        ),
        Drift(name="D", length=1.0, steps=1, map_steps=10, type_id=0, radius=1.0),
        WriteFull(
            name="final_particles",
            length=0.0,
            steps=0,
            file_id=101,
            type_id=-2,
            unused_2=0.0,
            sample_frequency=0,
        ),
    ],
    filename=None,
    verbose=False,
)

input.set_twiss_z(1e-12, 0.1e6, 0.5 * 0.1e6 * 1e-12)

In [ ]:
I1 = IZ.ImpactZ(input, use_temp_dir=False, workdir="./tmp")

In [ ]:
%%time

I1.run()
stats1 = I1.output.stats

In [ ]:
I2 = I1.copy()
I2.input.diagnostic_type = 2
I2.run()
stats2 = I2.output.stats

In [ ]:
P0 = I1.output.particles["initial_particles"]
P1 = I1.output.particles["final_particles"]

P0.plot("t", "energy")

# Ouput Comparison

In [ ]:
from pmd_beamphysics.units import mec2

from pmd_beamphysics.statistics import twiss_calc


def simple_twiss(pg: ParticleGroup, plane, p0c):
    cov = pg.cov(plane, "p" + plane)
    sigma = np.array(
        [[cov[0, 0], cov[0, 1] / p0c], [cov[0, 1] / p0c, cov[1, 1] / (p0c) ** 2]]
    )
    return twiss_calc(sigma)

In [ ]:
def compare(key, pg: ParticleGroup, output: ImpactZOutput, iz=0):
    """
    key = key in output.stats
    """
    fref = output.reference_frequency
    p0c = output.stats.p0c[iz]
    eref = output.stats.energy_ref[iz]

    # Key cleanup for ParticleGroup
    pfactor = 1
    if key == "sigma_z":
        pkey = "sigma_t"
        pfactor = c
    elif "gammabeta_x" in key:
        pkey = key.replace("gammabeta_x", "px")
        pfactor = 1 / pg.mass
    elif "gammabeta_y" in key:
        pkey = key.replace("gammabeta_y", "py")
        pfactor = 1 / pg.mass
    elif key.endswith("_over_p0"):
        pkey = key.replace("_over_p0", "")
        pfactor = 1 / p0c
    elif key == "max_r_rel":
        pkey = "max_delta_r"
    else:
        pkey = key

    # Get ParticleGroup value
    if key.startswith("max_abs_"):
        vp = np.max(np.abs(pg[pkey.replace("max_abs_", "")])) * pfactor

    elif key == "max_energy_dev":
        vp = np.max(np.abs(pg.energy - eref))

    elif key == "twiss_alpha_x":
        # vp = particle_twiss_dispersion(pg, 'x', p0c)["alpha_x"]
        vp = pg.twiss("x")["alpha_x"]  # do not include p0c, gives differences
    elif key == "twiss_beta_x":
        # vp = particle_twiss_dispersion(pg, 'x', p0c)["beta_x"]
        vp = pg.twiss("x")["beta_x"]
    elif key == "twiss_alpha_y":
        # vp = particle_twiss_dispersion(pg, 'y', p0c)["alpha_y"]
        # vp = simple_twiss(pg, 'y', p0c)["alpha"]
        vp = pg.twiss("y")["alpha_y"]
    elif key == "twiss_beta_y":
        # vp = particle_twiss_dispersion(pg, 'y', p0c)["beta_y"]
        # vp = simple_twiss(pg, 'y', p0c)["beta"]
        vp = pg.twiss("y")["beta_y"]
    elif key == "sigma_phase_deg":
        vp = pg["sigma_t"] * fref * 360

    elif key.startswith("norm_emit_") and key.endswith("percent"):
        plane = key[10]
        fraction = float(key[12:14]) / 100
        vp = pg.twiss(
            plane,
            fraction=fraction,
        )[f"norm_emit_{plane}"]

    elif key.startswith("moment3_"):
        pkey = pkey.replace("moment3_", "")
        x = pg[pkey]
        x0 = np.mean(x)
        vp = np.abs(np.mean((x - x0) ** 3)) ** (1 / 3) * pfactor
    elif key.startswith("moment4_"):
        pkey = pkey.replace("moment4_", "")
        x = pg[pkey]
        x0 = np.mean(x)
        vp = np.abs(np.mean((x - x0) ** 4)) ** (1 / 4) * pfactor

    else:
        vp = pg[pkey] * pfactor

    vi = getattr(output.stats, key)[iz]

    close = np.isclose(vp, vi)

    print(f"{key:40} {vp:20.10e} {vi:20.10e} {close}")


def compare_all(pg: ParticleGroup, output: ImpactZOutput, iz=0):
    klist = dir(output.stats)

    for k in klist:
        if k.endswith("_ref"):
            continue
        if k.endswith("_rel"):
            continue
        if k.startswith("loadbalance"):
            continue
        if k.startswith("to_"):
            continue
        if k.startswith("from_"):
            continue
        if "norm_emit_z" in k:
            continue

        if len(getattr(output.stats, k)) == 0:
            continue
        if k in (
            "charge_state_n_particle",
            "units",
            "z",
            "extra",
            "p0c",
            "twiss_alpha_z",
            "neg_mean_rel_energy",
        ):
            continue
        try:
            compare(k, pg, output, iz=-1)
        except Exception:
            print(f"{k:40}        ERROR")


compare("twiss_alpha_y", P1, I1.output, iz=-1)

In [ ]:
compare_all(P1, I1.output, iz=-1)

In [ ]:
compare_all(P1, I2.output, iz=-1)